# 第9章 質問応答

## 9.5 文書検索モデルとChatGPTを組み合わせる

### 9.5.1 検索モデルの準備

In [ ]:
!pip install datasets openai==0.27 tiktoken transformers[ja] faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
from transformers import pipeline

dataset_name = "llm-book/aio-passages-bpr-bert-base-japanese-v3"
passage_dataset = load_dataset(dataset_name, split="train")

encoder_model_name = "llm-book/bert-base-japanese-v3-bpr-question-aio"
encoder_pipeline = pipeline(
    "feature-extraction", model=encoder_model_name, device="cuda:0"
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4288198 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___parquet/llm-book--aio-passages-bpr-bert-base-japanese-v3-ff2436ef460125f5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


In [ ]:
import faiss
import numpy as np
from tqdm import tqdm

# インデックスの初期化
embed_size = encoder_pipeline.model.config.hidden_size
faiss_index = faiss.IndexBinaryIDMap2(
    faiss.IndexBinaryFlat(embed_size)
)

with tqdm(total=len(passage_dataset)) as pbar:
    i = 0
    for batch in passage_dataset.iter(batch_size=512):
        bs = len(batch["embeddings"])

        # 埋め込みをインデックスに適したdtypeへと変換
        batch_embeddings = np.array(
            batch["embeddings"], dtype=np.uint8
        )
        batch_indices = np.arange(i, i + bs, dtype=np.int64)

        # 埋め込みをインデックスに格納
        faiss_index.add_with_ids(batch_embeddings, batch_indices)

        pbar.update(n=bs)
        i += bs

100%|██████████| 4288198/4288198 [04:45<00:00, 15007.88it/s]


In [ ]:
import torch

def embed_questions(questions: list[str]) -> np.ndarray:
    """質問文を実数ベクトルに変換"""
    output_tensors = encoder_pipeline(questions, return_tensors="pt")
    embeddings = np.stack(
        [
            t.squeeze(0)[0].numpy().astype(np.float32)
            for t in output_tensors
        ]
    )
    return embeddings

def binarize_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """実数ベクトルをバイナリベクトルに変換"""
    # 0未満の値を0に、0以上の値を1に変換
    binary_embeddings = np.where(embeddings < 0, 0, 1)
    # バイト単位でまとめてuint8に変換
    packed_binary_embeddings = np.packbits(binary_embeddings, axis=-1)
    return packed_binary_embeddings

In [ ]:
q_embed = embed_questions(["日本で一番高い山は何？"])
binary_q_embed = binarize_embeddings(q_embed)
print(binary_q_embed)

[[141 210 243   2 153 138 188 209 212 205 166 253 146  26  12 135 215 208
    0 233 223  17  43   5 198 163   1 116 111  63 138 212 106 118 126  84
   31 166  23 194 190 164 201 165 139  83 132  46  86  11  97  24  81 227
   12  46 128 115  17  49  22 224 101  16 252 100 229  33  92  77 216 119
  176  62 252  97  16 110 203  36 195 208  32 170 138 159 255  83  83 122
   64 166 100 246  83  44]]


In [ ]:
scores, passage_ids = faiss_index.search(binary_q_embed, k=5)
print(f"スコア: {scores}")
print(f"パッセージID: {passage_ids}")

スコア: [[168 169 170 173 174]]
パッセージID: [[2222502 2783191 2879647 3581406 1149955]]


In [ ]:
print(passage_dataset[passage_ids[0]]["text"])

['日本の領土に占める山間部の割合はおよそ7割程度である。2020年現在、日本で一番高い山は富士山で、逆に一番低い山は日和山である。日本は新期造山帯に位置し、多くの火山が見られる。日本では昔から人々が山と共生する文化が培われてきた。それらの山は里山とよばれ、農村などでは山を共有地として村全体で管理し、薪をとったり土や山菜などを利用する目的で利用され、手入れをされてきた。しかし、明治時代になるとそれらの山の中には国有地にされてしまったものも多く存在する。また、日本の山の中には霊峰とよばれ、民間信仰の場にされた山も多くある。山の中には昔から金山や銀山として、近代では銅や石炭などを入手するため多くの鉱山が作られ、鉱毒や粉塵などが問題になった。特に近代において、山林の多くで木材を得るための大規模な伐採などが行われた。', '3000 m峰は、独立峰の富士山と御嶽山及び飛騨山脈(北アルプス)と赤石山脈(南アルプス)の山域に限られている。24番目に高い山は剱岳 (2999 m) で、3000 mにわずか1 m届いていない。標高3000 mを越える一帯は森林限界のハイマツ帯で、高山植物の群生地となっている。また多くの3000 m峰のハイマツ帯は、ライチョウ(雷鳥)の生息地となっている。', '三角点は一般的に眺望の利く場所に設置され一等三角点は970点余りに上るが、その中で風格のある山容、優れた眺望、高い知名度、さらに概ね標高1,000m以上で登りがいのある山が選定された。一等三角点の最高峰は、標高3,121 mの南アルプスの赤石岳である。三角点がその山の最高峰とは限らず、山頂に三角点より高い場所があり、その地点すなわち標高点や測定点を以て山の高さとしている例も少なくない(例:早池峰、御嶽山、金剛山)。また連山を形成している場合、最高峰でないピークに一等三角点が設置されている例もある(例:赤城山、穂高岳、阿蘇山)。特殊な例としては雲仙普賢岳のように、噴火により一等三角点が最高峰でなくなった事例もある。利尻岳中腹の長官山のように全く山頂とは離れた地点に一等三角点が設置されている例もある(山頂は二等三角点「利尻絶頂」)。', 'この項では世界最高峰と考えられていた山(せかいさいこうほうとかんがえられていたやま)について述べる。19世紀初頭までアンデスが世界で最も高い山脈だと考えられて

In [ ]:
passage_id = 0
print(np.unpackbits(faiss_index.reconstruct(passage_id)))

[1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0
 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 1 1 0 1 0
 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0
 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1
 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1
 0 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1
 0 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0
 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0
 1 1 0 1 0 1 1 1 1 0 1 0 0 1 0 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 1 1 1 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0
 0 0 0 0 1 1 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0
 0 1 1 1 1 0 0 1 1 1 1 1 

In [ ]:
import numpy as np
import torch

def retrieve_passages(
    questions: list[str],
    binary_k: int = 2048,  # リランキングするための候補の取得数
    top_k: int = 3,  # リランキング後に出力する最終的なパッセージ数
) -> list[list[str]]:
    """質問から関連するパッセージを取得"""
    # 質問をバイナリベクトルに変換する
    q_embed = embed_questions(questions)
    q_bin_embed = binarize_embeddings(q_embed)

    # バイナリベクトルを使用して検索する
    binary_k_scores, binary_k_p_ids = faiss_index.search(
        q_bin_embed, binary_k
    )

    batch_size = len(questions)
    embed_dim = q_bin_embed.shape[-1] * 8
    # インデックスからパッセージのベクトルを復元
    p_uint8_embed = [
        faiss_index.reconstruct(int(p_id))
        for p_id in binary_k_p_ids.flatten()
    ]
    # uint8からboolに変換
    p_bin_embed = np.vstack([np.unpackbits(e) for e in p_uint8_embed])
    # 型とシェイプを変換
    p_bin_embed = p_bin_embed.astype(np.float32).reshape(
        batch_size, binary_k, embed_dim
    )
    p_bin_embed = p_bin_embed * 2 - 1

    # 質問の実数ベクトルとパッセージのバイナリベクトルで再度スコアを計算する
    re_scores = np.einsum("ijk,ik->ij", p_bin_embed, q_embed)
    top_k_indices = np.argsort(-re_scores, axis=-1)[:, :top_k]
    top_k_p_ids = np.take_along_axis(
        binary_k_p_ids, top_k_indices, axis=-1
    )

    # top_kのテキストを整形して出力する
    retrieved_texts: list[list[str]] = []
    for p_ids in top_k_p_ids:
        formatted_texts = [
            f"タイトル：{passage_dataset[i]['title']}\n"
            f"本文：{passage_dataset[i]['text']}"
            for i in p_ids.tolist()
        ]
        retrieved_texts.append(formatted_texts)

    return retrieved_texts

In [ ]:
passages = retrieve_passages(["日本で一番高い山は何？"], top_k=3)[0]
for passage in passages:
    print(passage)

タイトル：日本の山
本文：日本の領土に占める山間部の割合はおよそ7割程度である。2020年現在、日本で一番高い山は富士山で、逆に一番低い山は日和山である。日本は新期造山帯に位置し、多くの火山が見られる。日本では昔から人々が山と共生する文化が培われてきた。それらの山は里山とよばれ、農村などでは山を共有地として村全体で管理し、薪をとったり土や山菜などを利用する目的で利用され、手入れをされてきた。しかし、明治時代になるとそれらの山の中には国有地にされてしまったものも多く存在する。また、日本の山の中には霊峰とよばれ、民間信仰の場にされた山も多くある。山の中には昔から金山や銀山として、近代では銅や石炭などを入手するため多くの鉱山が作られ、鉱毒や粉塵などが問題になった。特に近代において、山林の多くで木材を得るための大規模な伐採などが行われた。
タイトル：日本の山一覧 (3000m峰)
本文：3000 m峰は、独立峰の富士山と御嶽山及び飛騨山脈(北アルプス)と赤石山脈(南アルプス)の山域に限られている。24番目に高い山は剱岳 (2999 m) で、3000 mにわずか1 m届いていない。標高3000 mを越える一帯は森林限界のハイマツ帯で、高山植物の群生地となっている。また多くの3000 m峰のハイマツ帯は、ライチョウ(雷鳥)の生息地となっている。
タイトル：一等三角点百名山
本文：三角点は一般的に眺望の利く場所に設置され一等三角点は970点余りに上るが、その中で風格のある山容、優れた眺望、高い知名度、さらに概ね標高1,000m以上で登りがいのある山が選定された。一等三角点の最高峰は、標高3,121 mの南アルプスの赤石岳である。三角点がその山の最高峰とは限らず、山頂に三角点より高い場所があり、その地点すなわち標高点や測定点を以て山の高さとしている例も少なくない(例:早池峰、御嶽山、金剛山)。また連山を形成している場合、最高峰でないピークに一等三角点が設置されている例もある(例:赤城山、穂高岳、阿蘇山)。特殊な例としては雲仙普賢岳のように、噴火により一等三角点が最高峰でなくなった事例もある。利尻岳中腹の長官山のように全く山頂とは離れた地点に一等三角点が設置されている例もある(山頂は二等三角点「利尻絶頂」)。


### 9.5.2 検索結果のプロンプトへの組み込み

In [ ]:
import os

# 取得したAPIキーに置き換えてください
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from abc import ABCMeta, abstractmethod

class PromptMaker(metaclass=ABCMeta):
    """クイズ用プロンプトを作成するための抽象クラス"""

    @abstractmethod
    def run(self, questions: list[str]) -> list[str]:
        """プロンプトの作成（具体的な実装は継承先で行われる）"""
        pass

class RetrievalPromptMaker(PromptMaker):
    """
    質問から関連するテキストを取得し、
    それを組み入れたプロンプトを作成する
    """

    def __init__(self, binary_k: int = 2048, top_k: int = 3):
        """初期化処理の定義"""
        self.binary_k = binary_k
        self.top_k = top_k

    def run(self, questions: list[str]) -> list[str]:
        """プロンプトを作成"""
        retrieved_passages_list = retrieve_passages(
            questions, binary_k=self.binary_k, top_k=self.top_k
        )
        retrieved_texts = [
            "\n".join(ps) for ps in retrieved_passages_list
        ]
        return [
            "あなたには今からクイズに答えてもらいます。"
            "問題を与えますので、その解答のみを簡潔に出力してください。\n"
            "また解答の参考になりうるテキストを与えます。"
            "解答を含まない場合もあるのでその場合は無視してください。\n"
            "\n"
            f"{text}\n"
            "\n"
            f"問題：{question}\n"
            "解答："
            for question, text in zip(questions, retrieved_texts)
        ]

In [ ]:
retrieval_prompt_maker = RetrievalPromptMaker(top_k=3)
print(retrieval_prompt_maker.run(["日本で一番高い山は何？"])[0])

あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。
また解答の参考になりうるテキストを与えます。解答を含まない場合もあるのでその場合は無視してください。

タイトル：日本の山
本文：日本の領土に占める山間部の割合はおよそ7割程度である。2020年現在、日本で一番高い山は富士山で、逆に一番低い山は日和山である。日本は新期造山帯に位置し、多くの火山が見られる。日本では昔から人々が山と共生する文化が培われてきた。それらの山は里山とよばれ、農村などでは山を共有地として村全体で管理し、薪をとったり土や山菜などを利用する目的で利用され、手入れをされてきた。しかし、明治時代になるとそれらの山の中には国有地にされてしまったものも多く存在する。また、日本の山の中には霊峰とよばれ、民間信仰の場にされた山も多くある。山の中には昔から金山や銀山として、近代では銅や石炭などを入手するため多くの鉱山が作られ、鉱毒や粉塵などが問題になった。特に近代において、山林の多くで木材を得るための大規模な伐採などが行われた。
タイトル：日本の山一覧 (3000m峰)
本文：3000 m峰は、独立峰の富士山と御嶽山及び飛騨山脈(北アルプス)と赤石山脈(南アルプス)の山域に限られている。24番目に高い山は剱岳 (2999 m) で、3000 mにわずか1 m届いていない。標高3000 mを越える一帯は森林限界のハイマツ帯で、高山植物の群生地となっている。また多くの3000 m峰のハイマツ帯は、ライチョウ(雷鳥)の生息地となっている。
タイトル：一等三角点百名山
本文：三角点は一般的に眺望の利く場所に設置され一等三角点は970点余りに上るが、その中で風格のある山容、優れた眺望、高い知名度、さらに概ね標高1,000m以上で登りがいのある山が選定された。一等三角点の最高峰は、標高3,121 mの南アルプスの赤石岳である。三角点がその山の最高峰とは限らず、山頂に三角点より高い場所があり、その地点すなわち標高点や測定点を以て山の高さとしている例も少なくない(例:早池峰、御嶽山、金剛山)。また連山を形成している場合、最高峰でないピークに一等三角点が設置されている例もある(例:赤城山、穂高岳、阿蘇山)。特殊な例としては雲仙普賢岳のように、噴火により一等三角点が最高峰でなくなっ

In [ ]:
import tiktoken

def calculate_prompt_cost(
    prompts: list[str],
    num_output_tokens: int = 40,
    model: str = "gpt-3.5-turbo",
    usd_per_token: float = 0.0015 / 1000,
):
    """
    プロンプトをOpenAI APIに送信した際にかかる費用を見積もる
    """
    # トークナイザの初期化
    encoding = tiktoken.encoding_for_model(model)

    # 入力プロンプトの合計トークン数を算出
    total_num_prompt_tokens = 0
    for prompt in prompts:
        total_num_prompt_tokens += len(encoding.encode(prompt))

    avg_num_prompt_tokens = total_num_prompt_tokens / len(prompts)
    print(
        "入力プロンプトの平均トークン数:"
        f" {int(avg_num_prompt_tokens)}"
    )

    # モデル出力の合計トークン数を見積もる
    total_num_output_tokens = num_output_tokens * len(prompts)

    # 費用の計算
    total_cost = (
        total_num_prompt_tokens + total_num_output_tokens
    ) * usd_per_token
    print(f"合計コスト: {round(total_cost, 3)} USD")

In [ ]:
from tqdm import tqdm

quiz_dataset = load_dataset("llm-book/aio", split="validation")
prompts = []
with tqdm(total=len(quiz_dataset)) as pbar:
    for batch in quiz_dataset.iter(batch_size=8):
        prompts += retrieval_prompt_maker.run(batch["question"])
        pbar.update(len(batch["question"]))
calculate_prompt_cost(prompts)

入力プロンプトの平均トークン数: 1098
合計コスト: 1.708 USD


In [ ]:
import asyncio
from typing import Awaitable, Callable, TypeVar
from openai.error import OpenAIError

T = TypeVar("T")

async def retry_on_error(
    openai_call: Callable[[], Awaitable[T]],
    max_num_trials: int = 5,
    first_wait_time: int = 10,
) -> Awaitable[T]:
    """OpenAI API使用時にエラーが返ってきた場合に再試行する"""
    for i in range(max_num_trials):
        try:
            # 関数を実行する
            return await openai_call()
        except OpenAIError as e:
            # 試行回数が上限に達したらエラーを送出
            if i == max_num_trials - 1:
                raise
            print(f"エラーを受け取りました：{e}")
            wait_time_seconds = first_wait_time * (2**i)
            print(f"{wait_time_seconds}秒待機します")
            await asyncio.sleep(wait_time_seconds)

In [ ]:
import openai

async def _async_batch_run_chatgpt(
    messages_list: list[list[dict[str, str]]],
    temperature: float,
    max_tokens: int | None,
    stop: str | list[str] | None,
) -> list[str]:
    """OpenAI APIに並列してリクエストを送る"""
    # コルーチンオブジェクトをtasksに格納
    tasks = [
        retry_on_error(
            # ラムダ式で無名関数を定義して渡し、
            # retry_on_error関数の内部で呼び出させる
            openai_call=lambda x=ms: openai.ChatCompletion.acreate(
                model="gpt-3.5-turbo",
                messages=x,
                temperature=temperature,
                max_tokens=max_tokens,
                stop=stop,
            )
        )
        for ms in messages_list
    ]
    # tasks内の非同期処理を実行し結果を収集
    completions = await asyncio.gather(*tasks)
    return [
        c["choices"][0]["message"]["content"] for c in completions
    ]

def batch_run_chatgpt(
    messages_list: list[list[dict[str, str]]],
    temperature: float = 0.0,
    max_tokens: int | None = None,
    stop: str | list[str] | None = None,
) -> list[str]:
    """非同期処理関数を実行するためのラッパー"""
    return asyncio.run(
        _async_batch_run_chatgpt(
            messages_list, temperature, max_tokens, stop
        )
    )

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from datasets import Dataset
from tqdm import tqdm

def get_chatgpt_outputs_for_quiz(
    quiz_prompt_maker: PromptMaker,
    quiz_dataset: Dataset,
    batch_size: int,
) -> list[str]:
    """
    クイズ用のプロンプトを使用した際の
    データセットの各問題に対するモデルの解答を集める
    """
    output_answers: list[str] = []
    with tqdm(total=len(quiz_dataset)) as pbar:
        for batch in quiz_dataset.iter(batch_size=batch_size):
            # 入力の準備
            prompts = quiz_prompt_maker.run(batch["question"])
            inputs = [
                [{"role": "user", "content": p}] for p in prompts
            ]

            # APIにリクエストを送信
            answers = batch_run_chatgpt(inputs)

            # モデルの解答を表示
            for question, answer in zip(batch["question"], answers):
                print(f"問題：{question}")
                print(f"解答：{answer}")
                print()

            output_answers += answers
            pbar.update(len(answers))
    return output_answers

In [ ]:
def calculate_quiz_accuracy(
    output_answers: list[str], correct_answers_list: list[list[str]]
) -> float:
    """モデルの解答と正解の解答例から正解率を算出する"""
    num_correct = 0
    for output_answer, answers in zip(
        output_answers, correct_answers_list
    ):
        # モデルの出力が解答例を一つでも含んでいれば正解とみなす
        num_correct += int(any(a in output_answer for a in answers))
    return num_correct / len(output_answers)

In [ ]:
retrieval_output_answers = get_chatgpt_outputs_for_quiz(
    retrieval_prompt_maker, quiz_dataset, batch_size=4
)
retrieval_accuracy = calculate_quiz_accuracy(
    retrieval_output_answers,
    [item["answers"] for item in quiz_dataset],
)
print(f"正解率: {retrieval_accuracy * 100}")

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  0%|          | 4/1000 [00:02<10:20,  1.60it/s]

問題：映画『ウエスト・サイド物語』に登場する2つの少年グループといえば、シャーク団と何団?
解答：ジェット団

問題：氷った海に穴を開けて漁をすることから、漢字で「氷の下の魚」と書くタラ科の魚は何?
解答：コマイ

問題：アメリカとロシア間で結ばれた「戦略兵器削減条約」のことをアルファベット5文字の略称で何という?
解答：START

問題：リンゴが木から落ちる様子を見て「万有引力」を発見したといわれる、17世紀のイギリスの科学者は誰?
解答：アイザック・ニュートン



/usr/lib/python3.10/threading.py:852: RuntimeWarning: coroutine '_async_batch_run_chatgpt' was never awaited
  kwargs = {}
  1%|          | 8/1000 [00:04<09:43,  1.70it/s]

問題：聖武天皇の時代に栄えた、東大寺正倉院や唐招提寺金堂など、中国・唐の影響を強く受けた貴族文化を何という?
解答：天平文化

問題：アメリカ・メジャーリーグの2つのリーグといえば、ナショナルリーグと何リーグ?
解答：アメリカンリーグ

問題：池坊、草月流、小原流といえば、どんな分野の三大流派?
解答：いけばな

問題：英語で「取り巻く」という意味がある、前後左右から音が聞こえる立体音響を何という?
解答：サラウンドサウンド



  1%|          | 12/1000 [00:06<09:07,  1.81it/s]

...

問題：「冬の大三角」を形成する3つの星は、おおいぬ座のシリウス、こいぬ座のプロキオンとオリオン座の何?
解答：ベテルギウス

問題：江戸時代末期・大坂に洗心堂という私塾を開いた儒学者で、1837年に江戸幕府に対して反乱を起こすも鎮圧され自害した人物は誰?
解答：吉田松陰

問題：『ブエノスアイレス午前零時』で芥川賞を受賞したのは藤沢周ですが、『暗殺の年輪』で直木賞を受賞したのは誰?
解答：藤沢周平

問題：手紙やハガキを送った際に、料金を支払った証拠として郵便局で押される、日付入りのスタンプのことを何という?
解答：消印

エラーを受け取りました：The server is overloaded or not ready yet.
10秒待機します


100%|█████████▉| 996/1000 [12:27<00:13,  3.30s/it]

問題：NBAや格闘技などでみられる、選手が対戦相手に対して挑発的な言葉を発して心理的な揺さぶりをかける戦術を何という?
解答：トラッシュ・トーク

問題：いい加減で役立たずな人のことを、落語によく登場する間抜けな人物にたとえて何という?
解答：愚か者

問題：パーソン、パンサー、ジャクソンなどの種類がいる、周囲の環境に合わせて体の色を変えることでおなじみの動物といえば何?
解答：カメレオン

問題：小説『古都』や『伊豆の踊子』などの作品を書き、1968年にノーベル文学賞を受賞した作家は誰?
解答：川端康成



100%|██████████| 1000/1000 [12:28<00:00,  1.34it/s]

問題：いわゆる「古典の三大随筆」と呼ばれる3つの作品といえば、『枕草子』、『方丈記』と何?
解答：『徒然草』

問題：正式名称を「新幹線電気軌道総合試験車」という、新幹線の線路や架線の状態を調査するための車両を車体の色から何という?
解答：ドクターイエロー

問題：明治11年、東京・紀尾井坂で石川県士族の島田一郎らに暗殺された政治家は誰?
解答：大久保利通

問題：中国を代表するIT企業の総称「BATH」で表される4つの会社といえば、Baidu、Alibaba、Tencentと何?
解答：Huawei

正解率: 70.7


In [ ]:
quiz_example = quiz_dataset[87]
print("問題" + quiz_example["question"])
print("解答：" + str(quiz_example["answers"][0]) + "\n")

問題姥山貝塚、西之城貝塚、加曽利貝塚などをはじめ、全国にある貝塚の4分の1が位置する都道府県はどこ?
解答：千葉県



In [ ]:
prompt = retrieval_prompt_maker.run([quiz_example["question"]])[0]
answer = batch_run_chatgpt([[{"role": "user", "content": prompt}]])[0]
print(f"RetrievalPromptMakerを用いたモデルの解答：{answer}")

RetrievalPromptMakerを用いたモデルの解答：千葉県


In [ ]:
print(prompt)

あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。
また解答の参考になりうるテキストを与えます。解答を含まない場合もあるのでその場合は無視してください。

タイトル：水産試験場
本文：水産試験場(すいさんしけんじょう)は、水産に関する研究を行う機関。日本では、奈良県を除く46都道府県や市町村(市が運営する水産試験場は広島市と長崎市にある)、大学、研究機関・大規模な水産会社などが設置する。
タイトル：日本の貝塚一覧
本文：現代では海に面していない栃木県・群馬県・埼玉県にも縄文時代早期末から前期には海が入り込んでいた。(縄文海進)そのため、貝塚が存在する。。渡良瀬遊水地周辺には貝塚がある埼玉県には100ヶ所以上の貝塚がある。縄文海進でもっとも海面が高く奥東京湾が広がった縄文前期の物が多く、縄文中 - 後期の貝塚は前期よりは少なくなる。国指定史跡となっている貝塚も4つある。千葉県にはおよそ700ヶ所もの貝塚があり(近接している貝塚群を1つと数えるか複数で数えるかでも変わってくる)、日本の貝塚のおよそ1/4が千葉県にある。千葉県では12の貝塚が国指定史跡になっている。東京にはおよそ100ヶ所ほどの貝塚があるが、開発で湮滅し詳細不明な貝塚も少なくない神奈川県立歴史博物館編集『神奈川県貝塚地名表』2008年によると、神奈川県内には小規模な痕跡や歴史時代の物まで含めて292ヶ所の貝塚があるとされている。
タイトル：私立学校
本文：2013年(平成25年)5月1日現在私立小学校は日本全国に221校あるが、東京都53校、神奈川県31校、大阪府17校、兵庫県11校、京都府10校、千葉県9校、広島県8校、福岡県8校、奈良県6校と、南関東と近畿に集中している。その一方、私立小学校がない県も存在する。2013年(平成25年)5月1日時点、私立中学校は日本全国に771校あるが、東京都188校、大阪府65校、神奈川県64校、兵庫県41校、埼玉県29校、広島県28校、福岡県27校、静岡県27校、京都府26校、千葉県24校、愛知県22校、北海道16校と、南関東と近畿に集中している。しかし2012年(平成24年)5月1日現在において、東京都でも中学校第1学年の生徒数比率は25.7%と相対的に少数派である。私立高等学校は日本全国に1320校(この数値には